In [19]:
import requests
import pandas as pd
import datetime
import time
import random

In [20]:
from bs4 import BeautifulSoup
from dateutil.relativedelta import relativedelta
from concurrent.futures import ThreadPoolExecutor

In [29]:
def scrape_news(query, start_date, end_date):
    """
    Scrapes news articles from Google News RSS for a given query and filters them by data.

    Parameters:
        query(str): the stock or company name to search for.
        start_date(date): the start date for filtering articles.
        end_date(date): the end date for filtering articles.
        max_entries(int): number of times to retry incase of request failure.

    returns:
        pd.DataFrame: a dataframe consisting of article title, link, and publication date
    """
    articles = []
    url = f"https://news.google.com/rss/search?q={query}+after:{start_date}+before:{end_date}"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'xml')
    items = soup.find_all('item')

    for item in items:
        title = item.title.text
        link = item.link.text
        pub_date = item.pubDate.text
        articles.append({'title': title, 'link': link, 'pub_date': pub_date})
    return pd.DataFrame(articles)

In [30]:
def scrape_news_over_date_range(query, start_date, end_date):
    """
    Scrapes news articles over a given date range by iterating through months.

    Parameters:
        query(str): The stock or company name to search for.
        start_date(str): the start date in 'yyyy-mm-dd' format.
        end_date(str): the end date in 'yyyy-mm-dd' format.

    Returns:
        pd.DataFrame: A DataFrame collected across the date range.
    """
    start_date_dt = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()
    end_date_dt = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()

    all_articles = pd.DataFrame()

    current_date = start_date_dt

    while current_date<end_date_dt:
        next_date = min(current_date + relativedelta(months = 1), end_date_dt)
        
        start_str = current_date.strftime('%Y-%m-%d')
        end_str = next_date.strftime('%Y-%m-%d')
        month_articles = scrape_news(query, start_str, end_str)
        all_articles = pd.concat([all_articles, month_articles], ignore_index=True)
        current_date = next_date

    return all_articles    

In [31]:
def scrape_and_save_news(queries, start_date, end_date):
    """
    Scrapes news articles for mutliple queries over a gives date range and saves each result to a csv file

    Parameters:
        queries(list): A list of stock or company names to search for.
        start_date(str): the start date in 'yyyy-mm-dd' format.
        end_date(str): the end date in 'yyyy-mm-dd' format.

    Outputs:
        CSV files named after each query, containing scraped news articles.
    """
    for query in queries:
        articles = scrape_news_over_date_range(query, start_date, end_date)
        output_csv = f"{query}.csv"
        articles.to_csv(output_csv, index=False)
        print(f"Saved {query} articles to {output_csv}")

In [32]:
queries = ["Reliance", "Microsoft", "Google"]
start_date = '2020-01-01'
end_date = '2022-12-31'
scrape_and_save_news(queries, start_date, end_date)

Saved Reliance articles to Reliance.csv
Saved Microsoft articles to Microsoft.csv
Saved Google articles to Google.csv
